In [ ]:
import torch
from torch import nn
from data_loader import get_loader

In [2]:
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(SimpleAutoencoder, self).__init__()
        
        layers = []
        layers += [nn.Linear(input_dim, 200)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(200, 100)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(100, 50)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(50, 10)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(10, 5)]
        self.encoder = nn.Sequential(*layers)
        
        layers = []
        layers += [nn.Linear(5, 10)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(10, 50)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(50, 100)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(100, 200)]
        layers += [nn.ReLU(True)]
        layers += [nn.Linear(200, input_dim)]
        self.decoder = nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [103]:
timeframe = 5 * 24 * 60
data = get_loader(timeframe=timeframe)

In [104]:
# define input dimention
lr = 1e-3
ds = iter(data)
input_dim = next(ds).shape[2]
# inizialise all needed parameters
model = SimpleAutoencoder(input_dim=input_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [105]:
model

SimpleAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=50400, out_features=200, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU(inplace)
    (6): Linear(in_features=50, out_features=10, bias=True)
    (7): ReLU(inplace)
    (8): Linear(in_features=10, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=10, out_features=50, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=50, out_features=100, bias=True)
    (5): ReLU(inplace)
    (6): Linear(in_features=100, out_features=200, bias=True)
    (7): ReLU(inplace)
    (8): Linear(in_features=200, out_features=50400, bias=True)
  )
)

In [ ]:
epoch_num = 1
# model = model.float
for epoch in range(epoch_num):
    for i, x in enumerate(data):
        # forward
        model.train()
        x = x.float()
        y = model(x).float()
        loss = criterion(y, x)
#         MSE_loss = nn.CrossEntropyLoss()(y, x)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # ------- log output ---------
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, epoch_num, loss.data, 0))
   

In [ ]:
model.eval()
for i, x in enumerate(data):
    x = x.float()
    y = model(x[0])
    loss = criterion(y,x[0])
    print(criterion(y,x[0]))
    break;

In [ ]:
loss.data

In [ ]:
# test soll folgendes ausgeben
# 10 höchsten Werte mit Datum!


In [ ]:
import pandas as pd

In [ ]:
df_autoencoder = pd.read_csv('data/autoencoder_v1_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27', header=None)
df = pd.read_csv('data/PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv')

In [ ]:
print(df_autoencoder.shape)
print(df.shape)
dp = get_loader(batch_size=1, timeframe=60)

In [ ]:
result = []
for i, x in enumerate(dp):
    # add value to pandas 
#     x = x.unsqueeze(1)
    result.append(criterion(x.float(), model(x.float())).data.numpy())

In [ ]:
df_r = pd.DataFrame(result)

In [ ]:
df_r[0].nlargest(5)

In [ ]:
df_r.describe()

In [ ]:
df[513047:513047+1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_r.plot()
plt.show()

In [99]:
df_r[df_r > 1.5].dropna()

,0
513047,1.591862
513048,1.561770
513049,1.511238
513050,1.507709
513057,1.511746
513058,1.563944
513059,1.602898
513060,1.603214
513061,1.593110
513062,1.617410
